# Current News Scraper Development

<br>

### Imports

In [1]:
import pandas as pd

import requests
import xmltodict
from bs4 import BeautifulSoup

from IPython.core.display import JSON

<br>

### RSS Retrieval

Current News has an RSS feed that we can use to generate the JSON article definitions we require

In [20]:
def retrieve_all_current_articles():
    rss_url = 'https://www.current-news.co.uk/rss'

    r = requests.get(rss_url)
    xml = xmltodict.parse(r.content)

    articles = list(pd
                    .DataFrame(xml['rss']['channel']['item'])
                    .pipe(lambda df: df.assign(enclosure=df['enclosure'].apply(lambda x: x['@url'])))
                    .pipe(lambda df: df.assign(pubDate=pd.to_datetime(df['pubDate']).dt.strftime('%Y-%m-%d %H:%M')))
                    .rename(columns={
                        'link': 'article_url',
                        'description': 'lead',
                        'pubDate': 'date',
                        'source': 'section',
                        'enclosure': 'image_url'
                    })
                    .T
                    .to_dict()
                    .values()
                   )
    
    return articles

articles = retrieve_all_current_articles()

JSON(articles)

<IPython.core.display.JSON object>